In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

In [2]:
X_all = pd.read_csv('./data/train_x.csv')
y_all = pd.read_csv('./data/train_y.csv')
y_target = y_all.iloc[:,1]
X_test_submit = pd.read_csv('./data/test.csv')
X_train, X_test, y_train, y_test = train_test_split(X_all, y_target, test_size=0.3, random_state=0)

In [3]:
#list(X_all)

In [18]:
#_all['target'] = y_all.status_group
#en(y_all.status_group.value_counts())
#list_categorical_to_use
#X_all.dtypes

In [8]:
# filter categories that do not have too many different values
list_categorical_to_use = []
list_float_to_use = []
for i, i_type in enumerate(X_all.dtypes):
    if "object" == i_type and X_all.isnull().sum()[i]==0 and len(X_all[list(X_all)[i]].value_counts()) < 50:
        list_categorical_to_use.append(list(X_all)[i])
    if "float64" == i_type and X_all.isnull().sum()[i]==0:
        list_float_to_use.append(list(X_all)[i])
        
        

# float list 
#X_all.dtypes=='float64'
#y.status_group
#X_all.isnull().sum()

In [19]:
#list_categorical_to_use1
#_all.dtypes=='float64'
list_vars_to_use = list_float_to_use+list_categorical_to_use
X_to_use = X_all[list_vars_to_use]

In [20]:
#X_all.groupby('target').amount_tsh.mean()
def plot_bar_cross_target(X_all, name_var):
    """
    a function that plots the frequencey within each group
    """
    X_all.groupby(name_var).target.value_counts(normalize=True).unstack().plot.bar(stacked=True)
#for name_cat in list_categorical_to_use:
#    plot_bar_cross_target(X_all, name_cat)
#plot_bar_cross_target(X_all, "quality_group")

#X_all.construction_year
#construction_year
#X_all.head()
##plt.scatter(X_all[X_all.longitude!=0 && X_all.latitude!=0].longitude, X_all[X_all.longitude!=0 && X_all.latitude!=0].latitude)
#X_reg = X_all[['amount_tsh', 'gps_height', 'longitude', 'latitude', 'population', 'quantity_group', 'basin', 'extraction_type_class', 'payment_type', 'source_class', 'management_group', 'waterpoint_type_group']]
X_reg_transformed = pd.get_dummies(X_to_use)
#X_test_transformed = pd.get_dummies(X_train)
X_train, X_test, y_train, y_test = train_test_split(X_reg_transformed, y_target, test_size=0.3, random_state=0)

In [6]:
# fill 
#X_all.funder = X_all.funder.fillna("Unknown")
#X_all.lga.value_counts()

In [21]:
from sklearn.linear_model import LogisticRegressionCV

logistic_reg = LogisticRegressionCV()
logistic_reg.fit(X_train, y_train)

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0)

In [24]:
print("train error %s" % logistic_reg.score(X_train, y_train))
print("test error %s" %logistic_reg.score(X_test, y_test))
y_pred = logistic_reg.predict(X_test)

train error 0.731986531986532
test error 0.7319304152637486


In [30]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

                         precision    recall  f1-score   support

             functional       0.71      0.91      0.79      9612
functional needs repair       0.66      0.05      0.10      1333
         non functional       0.79      0.62      0.70      6875

            avg / total       0.74      0.73      0.70     17820



In [26]:
X_submit = X_test_submit[list_vars_to_use]
X_submit = pd.get_dummies(X_submit)
y_pred_submit = logistic_reg.predict(X_submit)

ValueError: X has 159 features per sample; expecting 160